<a href="https://colab.research.google.com/github/tamburins/ESAA_2023/blob/main/ESAA_5_15(579_590)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 05 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
이는 유명 영화 데이터 정보 사이트인 imdb의 영화중 주요 5000개 영화에 대한 메타정보를 새롭게 가공해 캐글에서 제공하는 데이터세트이다.

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
콘텐츠 기반 필터링은 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성 구성요소도등을 가진 다른 영화를 추천하는 것이다,

이렇게 영화 혹은 상품,서비스간 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠를 기반으로 하는 것이 바로 콘텐츠 기반 필터링이다.

콘텐츠 기반 필터링 추천 시스템을 영화를 선택하는데 중요한 요소인 영홪아르 속성을 기반으로 만들어보자. 장르 ㄱ칼러값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식이다.

### 데이터 로딩 및 가공

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('/content/drive/MyDrive/ESAA/OB/tmdb_5000_movies.csv')
print(movies)
movies.head(1)

         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


movies 데이터는 4803개의 레코드와 20개의 피처로 구성돼있다. 영화제목 개요 인기도 평점 투표수  예산 키워드 등 영화에 대한 다양한 메타정보를 가지며 이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 새롭게 데이터프레임으로 만든다. 추출할 주요 칼럼은 id, title, genre, vote_average, vote_count, popularity, keywords, overview이다.

In [5]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count', 'popularity','keywords', 'overview']]

칼럼 중 파이썬 리스트 내부에 여러개의 딕셔너리 형태의 문자열로 표기된 것이 있다. 이는 한꺼번에 여러개의 값을 표한하기 위한 것으로 가공을 요한다.

In [6]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


위오 ㅏ같이 장르는 여러개으 ㅣ개별 장르 데이터를 가지며 이 개별 장르의 명칭은 딕셔너리으 ㅣ키인 Name으로 추출할 수 있다. 키워드도 마찬가지의 구조로 장르 칼럼의 문자열을 분해해서 개별 장르를 파이선 리스트 객체로 추출하고자 한다. ast 파이선 모듈의 literal eval함수를 이용하면 이를 리스트 객체로 만들 수 있다.

In [ ]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

이제 장르칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 갖는다. 장르 칼럼에서 장르명만 리스트 객체로 추출해보자. 장르 칼럼에서 name에 해당하는 값을 추출하기 위해 apply 람다식을 이용한다.

In [ ]:
movies_df['genres']=movies_df['genres'].apply(lambda x:[y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

### 장르 콘텐츠 유사도 측정
여러개의 개별 장르가 리스트로 구성된 장르칼럼은 영화의 장르가 일부 동일할 경우 문자열로 변경한 뒤 countvectorizer로 피처벡터화한 행렬 데이터값을 코사인 유사도로 비교하는 것이다.

문자열로 변환된 장르 칼럼을 count 기반으로 피처벡터화 변환한다.

장르 문자열을 피처벡터화 행렬로 변환한 데이터세트를 코사인 유사도를 통해 비교한다. 이를 위해 데이터세트의 레콛별 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성한다.

장르 유사도가 높은 영화중 평점이 높은 순으로 영화를 추천한다.

먼저 피처벡터행렬화를 진행한다. 리스트 객체값을 ㅗ구성된 장르칼럼을 개별 요소를 공백문자로 구분하는 문자열로 변환해 별도 칼럼으로 저장한다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# for applying countvectorizer, transform string which can seperate word 
movies_df['genres_literal']=movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

countvecotizer로 변환해 피처벡터행렬을 생성했다. 이렇게 샏ㅇ성된 피처벡터행렬에 사이킷런의 코사인유사도를 계산한다. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

코사인 유사도 호출로 생성된 장르 sim 객체는 영화데이터프레임의 장르리터럴 칼럼을 피처벡터화한 행렬 데이터의 행별 유사도정볼르 가지며, 결국 영화데이터프레임의 행별 장르 유사도값을 가진다. 영화데이터프레임을 장르기준으로 콘텐츠기반 필터링을 수행하려면 영화데이터프레임의 개별 레코드에 대해 가장 장르유사도가 높은 순으로 다른 레코드를 추출해야한다.

장르심 객체의 기준행렬로 비교대상이 되는 행의 유사도값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출한다. 값 이 높은 순으로 정렬된 비교대상행의 유사도값이 아닌 비교대상 행의 위치인덱스이다.

argsort를 이용해보자.



In [ ]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

변환된 값이 의미하는 것은 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고 그다음으로 유사도가 높은 레코드를 제시하며,  가장 유사도가 낮은 레코드는 마지막이라는것이다.

이제각 레코드의 장르 코사인 유사도가 가장 높은 순으로 정렬된 타레코드의 위치인덱스값을 가진다. 이제 위치인덱스를 이ㅛㅇㅇ해 언제든지 특정 레코드와 코사인 유사도가 높은 다른 레코드를 추출할 수 있다.

### 장르 콘텐츠 필터링을 이용한 영화 추천

장르 유사도에 따라 영화를 추천하는 함수를 생성한다. 

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, topn=10):
  #인자로 입력된 영화데이터프레임에서 영화제목칼럼이 입력된 영화이름값의 데이터프레임 추출
  title_movie = df[df['title']==title_name]

  # title_named를 가진 데이터프레임의 인덱스 객체를 ndarray로 반환
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도순으로 topn개의 인덱스 추출
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  # 추출된 top n index 출력, 이는 2차원 데이터
  # 데이터프레임에서 인덱스로 사용하기 위해 1차원 행렬로 변경
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

함수를 이용해 영화 대부와 유사한 장르별 영화 10개를 추천해보자. 

In [ ]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

대부 2편이 가장 먼저 추천되었다 그 외의 좋은친구들도 대부와 비슷한 유형으로 이를 추천해야할 것이다. 하지만 대부를 좋아하는 고객에게 추천하기 이해하기 어려운 낯선 영화도 많다.

이번에는 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경한다. 영화의 평점정보인 vote_average를 이용한다. 평균점수는 왜곡될 수 있기 때문에 확인해보자.

In [ ]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:!0]

쇼생크탈출이나 대부같은명작보다 높은 순위에 이름도 들어본 적 없는 영화가 높은 평점으로 있는데 이들의 평가횟수는 작다. 
평가횟수에 대한 가중치가 부여된 평점방식을 사용한다. 

가중평점은 투표회수와 평점을 부여하기 위한 최소투표횟수, 개별 영화에 대한 평균평점, 전체 영화에 대한 평균평점으로 표현한다.

In [ ]:
C = movies_df['vote_average'].mean()
m=movies_df['vote_count'].quantile(0.6)
print('C: ', round(C,3), 'm :',round(m,3))

기존 평점을 새로운 가중평점으로 변경하는 함수를 생성하고 이를 이용해 새 평점정보를 생성한다. 데이터프레임의 레코드를 인자로 받아 레코드의 투표횟수와 투표평균칼럼 그리고 m,C를 이용해 레코드별 가중 평점을 반환한다.

In [ ]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_average']
  return ( (v/(v+m)*R)+( (m/(m+v)))*C)

movies['weighted_vote']=movies.apply(weighted_vote_average, axis=1)

In [ ]:
movies_df[['title','vote_average','weighted_vote', 'vote_count']].sort_values('weighted_vote',ascending=False)[:10]

잘 뽑힘

새 평점기준에 따라 영화를 추천하자. 장르 유사성이 높은 영화를 후보군으로 선정하고 가중평균이 넢은 순으로추천하자.

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title']==title_name]
  title_index = title_movie.index.values

  # topn에 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_index.reshape(-1)
  #except original indexes
  similar_indexes = similar_indexes[similar_indexes!=title_index]
  # weighted 투표수순으로 추출
  return df.ilcc[similar_indexes].sort_values('weighted_vote',ascending=False[:top+n])

similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average','weighted_vote']]

# better than before